In [ ]:
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def timer(func):
    def inner(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(f"{end - start:.3f} с")
        return result
    return inner

Завантанження та клінінг даних

In [ ]:
# === pandas ===
@timer
def load1(filepath):
    df = pd.read_csv(filepath, sep=";", na_values="?", low_memory=False)
    df.dropna(inplace=True)
    return df

# === numpy ===
@timer
def load2(filepath):
    types = [
    ("Date", "U10"),
    ("Time", "U8"),
    ("Global_active_power", "f8"),
    ("Global_reactive_power", "f8"),
    ("Voltage", "f8"),
    ("Global_intensity", "f8"),
    ("Sub_metering_1", "f8"),
    ("Sub_metering_2", "f8"),
    ("Sub_metering_3", "f8")
]
    arr = np.genfromtxt(
        filepath, delimiter=";", dtype=types, names=True,
        missing_values="?", filling_values=np.nan, encoding="utf-8"
    )
    num_cols = arr.dtype.names[2:]
    arr = arr[~np.any([np.isnan(arr[col]) for col in num_cols], axis=0)]
    
    return arr

In [ ]:
df = load1("data.txt")
arr = load2("data.txt")

2.082 с
10.624 с


In [ ]:
df.head()

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,16/12/2006,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,16/12/2006,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,16/12/2006,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
arr

Завдання №1. 
Записи, у яких Global_active_power > 5 кВт.

In [ ]:
@timer
def task1(df):
    # === pandas ===
    df_t1 = df[df["Global_active_power"] > 5]
    return df_t1
@timer
def task12(arr):
    # === numpy ===
    arr_t1 = arr[arr["Global_active_power"] > 5] 
    return df_t1, arr_t1
df_t1 = task1(df)
arr_t1 = task12(arr)

0.009 с
0.021 с


In [ ]:
df_t1.head()

In [ ]:
arr_t1

Завдання №2. Записи, де Voltage > 235 В.

In [ ]:

def task2(df, arr):
    # === pandas ===
    start = time.time()
    df_t2 = df[df["Voltage"] > 235]
    print(f"=== pandas === {time.time() - start:.3f} с")

    # === numpy ===
    start = time.time()
    arr_t2 = arr[arr["Voltage"] > 235]
    print(f"=== numpy === {time.time() - start:.3f} с")
    return df_t2, arr_t2
df_t2, arr_t2 = task2(df, arr)

In [ ]:
df_t2.head()

In [ ]:
arr_t2

Завдання №3. Записи, у яких сила струму лежить в межах 19-20 А, та, у яких пральна машина та холодильних
споживають більше, ніж бойлер та кондиціонер.

In [ ]:
# === Завдання 3 ===
def task3(df, arr):
    # === pandas ===
    start = time.time()
    df_t3 = df[(df["Global_intensity"] >= 19) & (df["Global_intensity"] <= 20)]
    df_t3 = df_t3[df_t3["Sub_metering_2"] > df_t3["Sub_metering_3"]]
    print(f"=== pandas === {time.time() - start:.3f} с")

    # === numpy ===
    start = time.time()
    cond1 = (arr["Global_intensity"] >= 19) & (arr["Global_intensity"] <= 20)
    cond2 = arr["Sub_metering_2"] > arr["Sub_metering_3"]
    arr_t3 = arr[cond1 & cond2]
    print(f"=== numpy === {time.time() - start:.3f} с")

    return df_t3, arr_t3
df_t3, arr_t3 = task3(df, arr)

In [ ]:
df_t3.head()

In [ ]:
arr_t3

Завдання №4. Середні величини всіх 3-х груп споживання електричної енергії з випадково вибраних 500 000 записів  



In [ ]:
def task4(df, arr):
    # === pandas ===
    start = time.time()
    sample = df.sample(n=500_000, replace=False)
    df_t4 = sample[["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]].mean()
    print(f"=== pandas === {time.time() - start:.3f} с")

    # === numpy ===
    start = time.time()
    random = np.random.choice(len(arr), size=500_000, replace=False)
    arr_t4 = {
        col: np.mean(arr[col][random])
        for col in ["Sub_metering_1", "Sub_metering_2", "Sub_metering_3"]
    }
    print(f"=== numpy === {time.time() - start:.3f} с")
    return df_t4, arr_t4
df_t4, arr_t4 = task4(df, arr)

In [ ]:
df_t4

In [ ]:
arr_t4

Завдання №5. Записи, які:
- після 18-00 споживають більше 6 кВт за хвилину в середньому
- у яких основне споживання електроенергії у вказаний проміжок часу припадає на пральну машину, сушарку, холодильник та освітлення
- обраний кожен третій результат із першої половини та кожен четвертий результат із другої половини.

In [ ]:
# === Завдання 5 ===
def task5(df, arr):
    # === pandas ===
    start = time.time()
    evening = df[df["Time"] >= "18:00:00"]
    macarena = evening[evening["Global_active_power"] > 6]
    combo = macarena[
        (macarena["Sub_metering_2"] > macarena["Sub_metering_1"]) &
        (macarena["Sub_metering_2"] > macarena["Sub_metering_3"])
    ]
    mid = len(combo) // 2
    df_t5 = pd.concat([combo[:mid][::3], combo[mid:][::4]])
    print(f"=== pandas === {time.time() - start:.3f} с")

    # === numpy ===
    start = time.time()
    cond_time = arr["Time"] >= "18:00:00"
    cond_power = arr["Global_active_power"] > 6
    combo = arr[cond_time & cond_power]
    cond_main = (
        (combo["Sub_metering_2"] > combo["Sub_metering_1"]) &
        (combo["Sub_metering_2"] > combo["Sub_metering_3"])
    )
    group = combo[cond_main]
    mid = len(group) // 2
    np_3rd = group[:mid][::3]
    np_4rd = group[mid:][::4]
    arr_t5 = np.concatenate([np_3rd, np_4rd])
    print(f"=== numpy === {time.time() - start:.3f} с")
    return df_t5, arr_t5
df_t5, arr_t5 = task5(df, arr)

In [ ]:
df_t5.head()

In [ ]:
arr_t5[:5]

### Результати експерименту

| №  | Опис завдання| pandas Час (с) | numpy Час (с) | Оцінка pandas | Оцінка numpy | Коментар|
|-|-|-|-|-|-|-|
|1| Завантаження та очищення даних | 2.046 | 10.402 | 5 | 2 | pandas значно швидше і простіше для очищення |
|2| Відбір Global_active_power > 5 кВт | 0.006 | 0.019 | 5 | 4 | pandas зручніше, numpy трохи повільніше |
|3| Відбір Voltage > 235 В | 0.115 | 0.220 | 4 | 4 | обидві структури показали подібну ефективність |
|4| Струм 19–20 А, sub_metering_2 > sub_metering_3 | 0.010 | 0.049 | 5 | 4 | pandas простіше для комбінованих умов |
|5| Середні значення 3-х sub_metering для 500 000 випадкових записів | 0.194 | 0.088 | 4 | 5 | numpy швидший при числовій агрегації |
|6| Фільтрація за часом, потужністю, приладами, з вибіркою кожного 3-го/4-го запису | 0.126 | 0.070 | 4 | 5 | numpy значно швидший у складних умовах |
